<a href="https://colab.research.google.com/github/DrAlexSanz/MC-calculations/blob/main/Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimate the probability of winning the lottery
### Or at least estimate the probability of winning something in the lottery

First calculate the expectancy. And later do a simulation to see if it holds.

## Lottery = Lotería de Navidad.

I'm doing this analysis based on tenths of numbers, which is what everyone buys. Source is in Wikipedia: [Lotería de Navidad](https://es.wikipedia.org/wiki/Sorteo_Extraordinario_de_Navidad#Premios)



The mathematical expectation is 14 euros. Considering that you pay 20 euros to play, that's not a good game for you. Calculations are here: [Expected win](https://github.com/DrAlexSanz/MC-calculations/blob/main/expectation.xlsx)

Now, let's check this. If I generate a thousand or a million lottery draws I should be able to reproduce the results and keep track of the gains.

* I will always play the same number, because many people do and I save a random generation.
* I will generate a random number to calculate the prizes that I get and see the accumulated value.

In [1]:
import numpy as np
import pandas as pd

from random import randint
from random import seed # To reproduce later


In [4]:
# Check that I can generate random numbers
seed(13)
print(randint(0, 99999), randint(0, 99999), randint(0, 99999), randint(0, 99999))

33948 38110 89818 89663


In [8]:
my_number = 17069 # This is 13*1313

In [19]:
# Do the loop of 1000 lottery draws (10 for speed)
# Then compare my number and see if I win anything.

n_draws = 10
win = []
spent = 20 * n_draws

for i in range(n_draws):
    gordo = randint(0, 99999)
    second = randint(0, 99999)
    third = randint(0, 99999)
    fourth = [randint(0, 99999) for k in range(2)] # do the multiple prizes as lists
    fifth = [randint(0, 99999) for k in range(8)]
    pedrea = [randint(0, 99999) for k in range(1794)]

    # And now the special prizes: Next numbers to the main prizes
    app_gordo = [randint(0, 99999) for k in range(2)] # Gordo +/-1
    app_second = [randint(0, 99999) for k in range(2)] # Second +/-1
    app_third = [randint(0, 99999) for k in range(2)] # Third +/-1

    # Matching the first 3 digits of main prizes
    hundredth_gordo = [randint(0, 99999) for k in range(99)] # First 3 digits of Gordo
    hundredth_second = [randint(0, 99999) for k in range(99)] # First 3 digits of Second
    hundredth_third = [randint(0, 99999) for k in range(99)] # First 3 digits of Third
    hundredth_fourth = [randint(0, 99999) for k in range(198)] # First 3 digits of the two fourth prizes

    # Matching the last 2 digits of main prizes
    l2_gordo = [randint(0, 99999) for k in range(999)] # Last 2 digits of Gordo
    l2_second = [randint(0, 99999) for k in range(999)] # Last 2 digits of Second
    l2_third = [randint(0, 99999) for k in range(999)] # Last 2 digits of Third

    # Reintegro: Money back if you match the last digit of the Gordo
    reintegro = [randint(0, 99999) for k in range(9999)] # Last digit of Gordo